# Recognize named entities on Twitter with LSTMs

In this workshop, you will use a recurrent neural network to solve Named Entity Recognition (NER) problem. NER is a common task in natural language processing systems. It serves for extraction such entities from the text as persons, organizations, locations, etc. In this task you will experiment to recognize named entities from Twitter.

For example, we want to extract persons' and organizations' names from the text. Than for the input text:

    Donald Trump is the president of the United States

a NER model needs to provide the following sequence of tags:

    B-PER I-PER   O O O O O   B-COUNTRY  I-COUNTRY

Where *B-* and *I-* prefixes stand for the beginning and inside of the entity, while *O* stands for out of tag or no tag. Markup with the prefix scheme is called *BIO markup*. This markup is introduced for distinguishing of consequent entities with similar types.

A solution of the task will be based on neural networks, particularly, on Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Libraries

For this task you will need the following libraries:
 - [Pytorch](https://pytorch.org/docs/stable/index.html) — an open-source software library for Machine Intelligence.
 - [Numpy](http://www.numpy.org) — a package for scientific computing.

Add tutorial link to Pytorch


Import libraries and download data

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from common.evaluation import precision_recall_f1

import sys
sys.path.append("..")
import common.workshop as workshop

workshop.download_ner_generation()

File ner/train.txt is already downloaded.
File ner/test.txt is already downloaded.
File ner/validation.txt is already downloaded.


### Setup execution device

Note: since this is hevy computational task, we need to use GPU, make sure that the cell below outputs 'cuda'

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Execution device:',device)

Execution device: cuda


In [30]:
!ls ./ner/

test.txt  train.txt  validation.txt


In [31]:
DATA_DIR="ner"

### Read file 

Read data from file and change replace users and urls with tokens

In [32]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token

            # your code
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

And now we can load three separate parts of the dataset:
 - *train* data for training the model;
 - *validation* data for evaluation and hyperparameters tuning;
 - *test* data for final evaluation of the model.

In [33]:

train_tokens, train_tags = read_data(DATA_DIR + '/train.txt')
validation_tokens, validation_tags = read_data(DATA_DIR + '/validation.txt')
test_tokens, test_tags = read_data(DATA_DIR + '/test.txt')


In [80]:
print (train_tokens[:2])

[['RT', '<USR>', ':', 'Online', 'ticket', 'sales', 'for', 'Ghostland', 'Observatory', 'extended', 'until', '6', 'PM', 'EST', 'due', 'to', 'high', 'demand', '.', 'Get', 'them', 'before', 'they', 'sell', 'out', '...'], ['Apple', 'MacBook', 'Pro', 'A1278', '13.3', '"', 'Laptop', '-', 'MD101LL/A', '(', 'June', ',', '2012', ')', '-', 'Full', 'read', 'by', 'eBay', '<URL>', '<URL>']]


Lest take a look at our data

In [79]:

for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()


RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Prepare dictionaries

To train a neural network, we will use two mappings: 
- {token}$\to${token id}: address the row in embeddings matrix for the current token;
- {tag}$\to${tag id}: one-hot ground truth probability distribution vectors for computing the loss at the output of the network.

Now you need to implement the function *build_dict* which will return {token or tag}$\to${index} and vice versa. 

In [36]:
from collections import defaultdict

def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens (or tags) to indices and vice versa.
    # Add special tokens (or tags) to the dictionaries.
    # The first special token must have index 0.
    
    # Mapping tok2idx should contain each token or tag only once. 
    # To do so, you should extract unique tokens/tags from the tokens_or_tags variable
    # and then index them (for example, you can add them into the list idx2tok
    # and for each token/tag save the index into tok2idx).
    
    idx = 0

    # your code
    
    return tok2idx, idx2tok

In [37]:

special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)


The next additional functions will help you to create the mapping between tokens and ids for a sentence. 

In [38]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

### Generate batches

Neural Networks are usually trained with batches. It means that weight updates of the network are based on several sequences at every single time. The tricky part is that all sequences within a batch need to have the same length. So we will pad them with a special `<PAD>` token. It is also a good practice to provide RNN with sequence lengths, so it can skip computations for padding parts.

In [111]:

# This function generates batches of input data.
# Since each train example has variable length we need to pad some of them to make sure
# that each train data vector has the same size
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = len(tokens)
    
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    # number of batches
    n_batches = n_samples // batch_size
    # the last batch has smaller length, if our network can handle different batch sizes, we can also utilize the last batch
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1
    
    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        
        current_batch_size = batch_end - batch_start
        
        x_list = []
        y_list = []
        
        # max_len_token will be the length of the input sequence for each train example
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Initialize x as matrix of idx('<PAD>')
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        # Initialize y as matrix of idx('O')
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        
        # Iterate through batch size and assign variable sequence to each train example.
        # E.g. x[n, :m] = arr (arr should have length of m) - means: 
        # n'th row first m characters will be assigned values 
        for current_batch_num in range(current_batch_size):
            curr_example_length = len(x_list[current_batch_num])
            x[current_batch_num, :curr_example_length] = x_list[current_batch_num]
            lengths[current_batch_num] = curr_example_length
            y[current_batch_num, :curr_example_length] = y_list[current_batch_num]
            
        yield x, y, lengths


In [112]:
# check the generator

batch_size= 10
x,y, _ = next(batches_generator(batch_size, train_tokens, train_tags))

print(x.shape,y.shape)

(10, 23) (10, 23)


## Build a recurrent neural network

This is the most important part of the assignment. Here we will specify the network architecture based on Pytorch building blocks. It's fun and easy as a lego constructor! We will create an LSTM network which will produce probability distribution over tags for each token in a sentence. To take into account both right and left contexts of the token, we will use Bi-Directional LSTM (Bi-LSTM). Dense layer will be used on top to perform tag classification.  

In [41]:
## TODO: explain what architecture needs to be built, and provide documentation to corresponding blocks

import torch.nn as nn

class BiLstm(nn.Module):
    def __init__(self, vocab_size, embed_size, n_hidden, n_output):
        # Invoke the constructor
        super(BiLstm, self).__init__()
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        # simple embedding layer, that will be learned
        self.embed_layer = nn.Embedding(vocab_size, embed_size)
        
        # bidirectional 2 layer LSTM 
        self.lstm_layer = nn.LSTM(embed_size, n_hidden,
                                  num_layers = 2, batch_first = True, 
                                  bidirectional = True)
        
        # linear layer to produce outputs
        self.linear_layer = nn.Linear(2*n_hidden, n_output)
        

    # input_tensor - shape (batch_size, seq_length)
    # hidden - pair of tensors of shape (batch_size, hidden_size)
    def forward(self, input_tensor, seq_length, batch_size):

        # e_tensor - (batch_size, seq_length, embed_size)
        e_tensor = self.embed_layer(input_tensor)
              
        # execute lstm layer
        lstm_out, _ = self.lstm_layer(e_tensor)

        
        # transfor output to the 2d matrix of shape (batch_size * seq_length, 2*hidden_size)
        # since it is bidirectional network there is double size of hidden parameters
        output_tensor = lstm_out.contiguous().view(-1, 2 * self.n_hidden)
        # execute linear layer
        output_tensor = self.linear_layer(output_tensor)
        
        return output_tensor
        


In [42]:

# Constructs pytorch tensor from numpy array
def construct_pytorch_tensor(numpy_tensor):
    tensor = torch.from_numpy(numpy_tensor).long()
    # Send tensor to the device
    tensor = tensor.to(device)
    return tensor


def construct_model(vocab_size, embed_size, hidden_size, output_size):
    bi_nn = BiLstm(vocab_size=vocab_size, embed_size=embed_size, 
                 n_hidden=hidden_size, n_output=output_size)
    bi_nn = bi_nn.to(device)
    return bi_nn


### Test the network

lest test the created network. Play with parameters to see how do the affect input and output tensors

In [43]:
import numpy as np
import torch
import pdb

vocab_size = len(idx2token)
n_tags = len(idx2tag)
n_input = 100
n_hidden = 40
batch_size = 4
embed_size = 30

x,y, l = next(batches_generator(batch_size, train_tokens, train_tags))

# seq length
model_seq_length = x.shape[1]

input_tensor = construct_pytorch_tensor(x) # construct input tensor

target_tensor = construct_pytorch_tensor(y) # construct target tensor

bi_nn = construct_model(vocab_size, embed_size, n_hidden, n_tags) # init model

output_tensor = bi_nn(input_tensor, model_seq_length, batch_size) # execute forward
print(output_tensor.shape)




torch.Size([128, 21])


### Evaluation

below defined evaluation functions.

We use precision/recall metric and F1 score to determine the performance.

Additional resources:

[precision/recall](https://en.wikipedia.org/wiki/Precision_and_recall)

[f1score](https://skymind.ai/wiki/accuracy-precision-recall-f1)


In [85]:

def evaluate_on_data(model, tokens, tags):
    y_true_indx = []
    y_pred_indx = []
    batch_size = 32
    
    for i, (x, y, _) in enumerate(batches_generator(batch_size, tokens, tags)):
        input_tensor = torch.from_numpy(x).long().to(device)
        taget_tensor = torch.from_numpy(y).long().to(device)
        
        batch_size = x.shape[0]
        seq_length = x.shape[1]

        output_tensor = model(input_tensor, seq_length, batch_size)
        output_tensor = F.softmax(output_tensor, dim = 1)
        _, output_inds = output_tensor.max(dim = 1)
        
        output_inds = output_inds.long()
        
        y_true_indx_batch = list(taget_tensor.cpu().numpy().reshape(-1))
        y_pred_indx_batch = list(output_inds.cpu().detach().numpy().reshape(-1))
        y_true_indx = y_true_indx + y_true_indx_batch
        y_pred_indx = y_pred_indx + y_pred_indx_batch
        
    y_true = [idx2tag[idx] for idx in y_true_indx]
    y_pred = [idx2tag[idx] for idx in y_pred_indx]
    
    precision_recall_f1(y_true, y_pred, print_results=True, short_report=True)
    
def evaluate(model):
    # For evaluation we do not need to update gradients and compute derivatives
    with torch.no_grad():
        print('Evaluation on train set')
        evaluate_on_data(model, train_tokens, train_tags)
        print('Evaluation on validation set')
        evaluate_on_data(model, validation_tokens, validation_tags)


## Train loop

Below defined the train loop for a single epoch

In [45]:

def train(model, optimizer, loss_fn, batch_size = 32):
    for batch_num, (input_data, target_data, _) in enumerate(batches_generator(batch_size, train_tokens, train_tags)):
        
        # The last batch can be smaller than others
        train_batch_size = input_data.shape[0]
        # Since each batch has different sequence length we need to update the variable for each batch
        train_seq_length = input_data.shape[1]

        input_tensor = construct_pytorch_tensor(input_data)
        target_tensor = construct_pytorch_tensor(target_data)
        
        # zero out the gradients
        optimizer.zero_grad()
        
        # get the output sequence from the input and the initial hidden and cell states
        output_tensor = model(input_tensor, train_seq_length, train_batch_size).to(device)
    
        # we need to calculate the loss across all batches, so we have to flat the targets tensor
#         pdb.set_trace()
        target_tensor = target_tensor.view((train_seq_length*train_batch_size, -1)).squeeze(dim=1)
        
        # calculate the loss
        loss = loss_fn(output_tensor, target_tensor)

        # calculate the gradients
        loss.backward()
        
        # update the parameters of the model
        optimizer.step()


### Main train loop

We will be bu using precision/recall metric and F1 score as evaluation. Precision/recall is very useful metric that shows how well classification task performs. We can split our classification results on four categories:

    Lest say that our target_output is the true output of data, while predicted_output is the predicted output of our network. In this case:

    True positives - predicted_output that are labeled as positives, and target_output are actually positive
    True negatives - predicted_output that are labeled as negatives, and target_output are actually negatives
    False positives - predicted_output that are labeled as negatives, and target_output are actually positives
    False negatives - predicted_output that are labeled as positives, and target_output are actually negatives
    
    wheer positives - predicted sucsesfully, negatives - predicted unsucsessfully
    
In this case, precision and recall are defined as:

![title](images/precision_recall.png)

We also can combine these two metrics into a single one, called F1 score:

![title](images/f1-score.jpg)


Additional materials:

[PrecisionRecall](https://towardsdatascience.com/beyond-accuracy-precision-and-recall-3da06bea9f6c)

In [46]:
# number of epoch to train
n_epoch = 10

# size of the hidden layer
hidden_size = 256

# batch size of input
batch_size = 32

# embedding size
embed_size = 256

# size of output vector on each timestamp
n_tags = len(idx2tag)

# vocabulary size, it is needed to build embedding layer
vocab_size = len(idx2token)


model = construct_model(vocab_size, embed_size, hidden_size, n_tags) # init model


optimizer = torch.optim.Adam(model.parameters(), lr=0.005) # Adam optimizer

loss_function = nn.CrossEntropyLoss() # Cross entropy loss

for epoch in range(n_epoch):
    print('Starting epoch: ', epoch)
    train(model, optimizer, loss_function, batch_size)
    
    evaluate(model)


Starting epoch:  0
Evaluation on train set
processed 180520 tokens with 4489 phrases; found: 1021 phrases; correct: 663.

precision:  64.94%; recall:  14.77%; F1:  24.07

Evaluation on validation set
processed 22328 tokens with 537 phrases; found: 98 phrases; correct: 56.

precision:  57.14%; recall:  10.43%; F1:  17.64

Starting epoch:  1
Evaluation on train set
processed 179496 tokens with 4489 phrases; found: 4038 phrases; correct: 2525.

precision:  62.53%; recall:  56.25%; F1:  59.22

Evaluation on validation set
processed 22248 tokens with 537 phrases; found: 365 phrases; correct: 153.

precision:  41.92%; recall:  28.49%; F1:  33.92

Starting epoch:  2
Evaluation on train set
processed 178815 tokens with 4489 phrases; found: 4635 phrases; correct: 3432.

precision:  74.05%; recall:  76.45%; F1:  75.23

Evaluation on validation set
processed 22512 tokens with 537 phrases; found: 473 phrases; correct: 162.

precision:  34.25%; recall:  30.17%; F1:  32.08

Starting epoch:  3
Evalua

KeyboardInterrupt: 

Now lets try to use our model to some arbitrary data!

In [116]:
# add your sentencies

sentencies = ['Donald Trump is the president of United States']

my_tokens = []
my_tags = []

for sentence in sentencies:
    token_arr = sentence.split(' ')
    my_tokens.append(token_arr)
    my_tags.append(['O']*len(token_arr))

print(my_tokens)

for idx, (input_x,_,_) in enumerate(batches_generator(1, my_tokens, my_tags, shuffle= False)):
    sentence = sentencies[idx]
    input_tensor = torch.from_numpy(input_x).long().to(device)
    batch_size = input_x.shape[0]
    seq_length = input_x.shape[1]
    output_tensor = model(input_tensor, seq_length, batch_size)

    output_tensor = F.softmax(output_tensor, dim = 1)
    _, output_inds = output_tensor.max(dim = 1)

    output_inds = output_inds.long()

    print('Input: ', sentence)
    print('predicted tags: ', [idx2tag[ind] for ind in output_inds])



[['Donald', 'Trump', 'is', 'the', 'president', 'of', 'United', 'States']]
Input:  Donald Trump is the president of United States
predicted tags:  ['B-person', 'I-person', 'O', 'O', 'B-facility', 'O', 'B-sportsteam', 'I-geo-loc']


### Conclusions

So far we learned how to build mini batches, how to use word embeddings and how to use bidirectional RNNs. 
We reached decens score, but we always can do better. One improvement that can be done is to modify the loss function to not count padded tokens. But this will be left as a take home exercise.


But this is just the begginnig and there is more to learn. The good places to start are:

[fastai](https://www.fast.ai/)

[AdvancedMLSpecialization](https://www.coursera.org/specializations/aml)

[DeepLearningAI](https://www.deeplearning.ai/)
